<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/04_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **単語ベクトル**
* 文書をbag-of-wordsモデルによってベクトル表現することは、最近はあまり行われない。
* まず単語のベクトル表現を得て、それを使って文書のベクトル表現を得る、という手順をとる。
 * こうなったのは、[word2vec](https://arxiv.org/abs/1301.3781)と呼ばれる手法が登場して以降。
  * https://en.wikipedia.org/wiki/Word2vec

## spaCyの単語ベクトル
* 今回は英語テキストのみ。
* 小規模のモデル（名前が__`_sm`__で終わるモデル）は単語ベクトルを含まない。
* 大規模モデルはダウンロードに時間がかかる。
* そのため、中規模モデルをインストールする。

### spaCyの中規模モデルをダウンロード
* https://spacy.io/models/en#en_core_web_md 

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

### テキストをtokenizeする

In [ ]:
tokens = nlp("Dogs and cats have afskfsd")

In [ ]:
for token in tokens:
  print((f'単語:{token.text}, ベクトルの有無:{token.has_vector},'
        f'ベクトルのL2ノルム:{token.vector_norm:.4f}, ベクトルがOoVか否か:{token.is_oov}'))

* 単語ベクトルの型と中身を確認

In [ ]:
type(tokens[0].vector)

In [ ]:
tokens[0].vector

In [ ]:
tokens[0].vector.shape

* OoV(Out of Vocabulary)の単語ベクトルはゼロベクトル

In [ ]:
print(f'{tokens[-1].text}\n{tokens[-1].vector}')

### トークン列のベクトル表現を得る

In [ ]:
tokens.vector

* トークン列のベクトルを求めると、OoVのゼロベクトルも含めて平均が計算されるようだ。


In [ ]:
import numpy as np
np.allclose(tokens[:-1].vector * (len(tokens) - 1), tokens.vector * len(tokens))

* 多義語であっても単語ベクトルはひとつだけ
 * 意味の数だけ別々のベクトルが用意されていたりはしない。
* 例：社名のアップルであろうと、りんごのアップルであろうと、単語ベクトルは同一
 * `similarity`メソッドで、単語ベクトルどうしのコサイン類似度を計算できる。

In [ ]:
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))

### 文書類似度の計算

In [ ]:
doc1 = nlp("It's a warm summer day")
doc2 = nlp("It's sunny outside")
doc3 = nlp("It's definitely cold outside")

print(doc1.similarity(doc2))
print(doc1.similarity(doc3))
print(doc2.similarity(doc3))

# 単語ベクトルを使った文書分類
* 文書に含まれる単語の単語ベクトルから文書のベクトル表現を得る。
* 文書のベクトル表現を使って2値分類問題を解く。
* sentiment analysisの有名なデータセットであるIMDbを使う。

### IMDbデータセット

* データセットの基本情報
 * Webサイト: https://ai.stanford.edu/~amaas/data/sentiment/
 * 作成者: Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng and Christopher Potts
 * タイトル: Large Movie Review Dataset (aka. IMDb Review Dataset)
 * 公開日: Jun, 2011

* データのロード
 * ml-datasetsというツールを使う。
 * https://pypi.org/project/ml-datasets/

In [ ]:
!pip install ml_datasets

In [ ]:
from ml_datasets import imdb
train_data, test_data = imdb()

In [ ]:
print(f'{len(train_data)} {len(test_data)}')

In [ ]:
train_texts, train_labels = zip(*train_data)
test_texts, test_labels = zip(*test_data)

In [ ]:
train_texts[0]

In [ ]:
print(train_labels)

* spaCyを使ってIMDbデータセットの全文書をベクトル化してみる
 * 1,000件でも結構時間がかかる…。

In [ ]:
from tqdm import tqdm

X_train = []
for text in tqdm(train_texts[:1000]):
  X_train.append(nlp(text).vector)
X_train = np.array(X_train)

## fasttextの単語ベクトルを使う
* spaCyの単語ベクトルは、Pythonで実装されているので、遅い。
* fasttextは、C++で実装されているので、速い。
 * しかし、単語ベクトルデータのサイズが巨大なので、Google Colab向きではない。

### fasttextのインストール

In [ ]:
!pip install fasttext

### 言語モデルのダウンロード
* 7GB強のサイズがあるので、非常に時間がかかる。
 * こういう作業はGoogle Colabでは行わないほうが良いかも。
* 今回は、諦める。（試したい方は試してください。）
* 手元の環境で実行する際の手順だけを、以下のセルに示す。

In [ ]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')

### 文書のベクトル化
* モデルをロードしてから、ベクトル化。

In [ ]:
model = fasttext.load_model('cc.en.300.bin')

In [ ]:
import numpy as np

X_train = []
for text in train_texts:
  X_train.append(model.get_sentence_vector(text.replace("\n"," ")))
X_train = np.array(X_train)

X_test = []
for text in test_texts:
  X_test.append(model.get_sentence_vector(text.replace("\n"," ")))
X_test = np.array(X_test)

* ベクトル化した結果をファイルとして保存

In [ ]:
with open('train.npy', 'wb') as f:
  np.save(f, X_train)
with open('test.npy', 'wb') as f:
  np.save(f, X_test)

In [ ]:
with open('train_labels.npy', 'wb') as f:
  np.save(f, np.array(train_labels))
with open('test_labels.npy', 'wb') as f:
  np.save(f, np.array(test_labels))

## 事前にfasttextでベクトル化されたIMDbデータをロード
 * 下記のリンク先にある`.npy`ファイルを、あらかじめ自分のGoogle Driveの適当な場所に置いておく。
  * https://drive.google.com/drive/folders/1wSoIzSbZ2UqGQowiVDBI20h_A3hQNbtV?usp=sharing

In [ ]:
import numpy as np

PATH = '/content/drive/MyDrive/2022Courses/nlp/imdb/'
texts = dict()
labels = dict()
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
texts['train'][0]

## 事前学習済みTransformerで文書をベクトル化
* Transformerの説明はしない。とりあえず使う。
* Transformerを単なるエンコーダとして使う。
 * fine tuningはしない。
* 今回は、sentence transformersを使う。
 * https://github.com/UKPLab/sentence-transformers

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
apple1 = model.encode("Apple shares rose on the news.")
apple2 = model.encode("Apple sold fewer iPhones this quarter.")
apple3 = model.encode("Apple pie is delicious.")

In [ ]:
apple1

In [ ]:
apple1.shape

In [ ]:
from sentence_transformers import util

print(util.cos_sim(apple1, apple2))
print(util.cos_sim(apple1, apple3))

* IMDbの全文書のベクトル化には、やはりそれなりに時間がかかる。

In [ ]:
import tqdm

X_train = []
for text in tqdm.tqdm(train_texts[:100]):
  doc = model.encode(text)
  X_train.append(doc)

# 課題3
* 春学期に習った分類手法を使って、IMDbデータセットの感情分析をしてみよう。
 * training set / test setの分割は、そのまま使う。
 * training setをどのように使うかはお任せします。（交差検証など。）
 * test setでの分類性能をArea under the ROC curveで報告。